# Define Paths
This isn't necessary. It was something I started doing so to easily switch between different versions of the model


In [1]:
PROJECT_ROOT_DIR = '/home/rharshal/myProjects/thesis/deepspeech2-prediction'

CUSTOM_FILES_DIR = f'{PROJECT_ROOT_DIR}/src'

DATA_DIR = f'{PROJECT_ROOT_DIR}/data/'
MODEL_DIR = f'{PROJECT_ROOT_DIR}/ml-models/walltrainedon1305'


In [2]:
model_weights_path = f'{MODEL_DIR}/wallmodel1305.h5'
feature_mean_path = f'{MODEL_DIR}/mean-wall1305.pickle'
feature_stddev_path = f'{MODEL_DIR}/std-wall1305.pickle'

# Perform Imports

In [3]:
import os, sys
sys.path.append(CUSTOM_FILES_DIR)

In [4]:
from create_model import final_model
from data_generator import AudioGenerator
from utils import int_sequence_to_text

from keras import backend as K
from keras.layers import GRU

import pickle

# Load Preprocessing objects

In [5]:
with open(feature_mean_path, 'rb') as fp:
    feat_mean = pickle.load(fp)

with open(feature_stddev_path, 'rb') as fp:
    feat_std = pickle.load(fp)

# Load Model

In [6]:
model = final_model(input_dim=161,
                        filters=200,
                        kernel_size=11, 
                        conv_stride=2,
                        conv_border_mode='same',
                        units=250,
                        # activation='relu',
                        cell=GRU,
                        dropout_rate=0.2,
                        number_of_layers=2,
                        conv_layers=2)
                        

In [7]:
model.load_weights(model_weights_path)

# Do data files loading related stuff

In [8]:
def normalize_test(feature,mean,std,eps=1e-14):
    return (feature - mean) / (std + eps)

## Save file-names for easier re-trials

In [9]:
audio_filenames = [
    'boliye_1574414678229.wav',  'kitne_1574413891009.wav', 'nahimila_1574415020378.wav', 'order_1574414227763.wav',
    'kamhai_1574414938995.wav', 'milgaya_1574415168217.wav', 'number_1574414716726.wav', 'test.wav',
    'my_kitne.wav', 'my_boliye.wav', 'my_kamhai.wav', 'my_milgaya_1.wav', 'my_milgaya_2.wav']

audio_path = f'{DATA_DIR}/some-files/{audio_filenames[-4]}'
print(audio_path)

/home/rharshal/myProjects/thesis/deepspeech2-prediction/data//some-files/my_boliye.wav


## Create data generator class

In [10]:
data_gen = AudioGenerator(spectrogram=True)

## Load a file from the above list

In [11]:
data_point = data_gen.featurize(audio_path)
data_point_normalized = normalize_test(data_point,feat_mean, feat_std)
data_point_normalized.shape

(175, 161)

# Do Prediction

In [12]:
# Pass the spectrogram through the model
pred = model.predict(np.expand_dims(data_point_normalized, axis=0))
print(f'model output shape: {pred.shape}')

# Get additional info about the input, used for CTC-decoding below
output_length = [model.output_length(data_point.shape[0])] 
print(f'{output_length}')

# Do CTC decoding
pred_ints = (K.eval(K.ctc_decode(pred, output_length)[0][0])+1).flatten().tolist()
pred_l = int_sequence_to_text(pred_ints)
pred_text=''.join(pred_l)

print(f'Model output decoded: {pred_text}')

model output shape: (1, 44, 29)
[44]
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Model output decoded: boliye
